<a href="https://colab.research.google.com/github/LibertyHeise/deep-learning-challenge/blob/main/Charity_model_two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import matplotlib as plt
import sklearn as skl
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

application_df = pd.read_csv("/content/drive/MyDrive/ChariltyProj/charity_data.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
clean_df = application_df.drop(columns=['EIN','NAME'])



In [ ]:
# Determine the number of unique values in each column.
print(clean_df.isnull().sum())
clean_df.nunique()

APPLICATION_TYPE          0
AFFILIATION               0
CLASSIFICATION            0
USE_CASE                  0
ORGANIZATION              0
STATUS                    0
INCOME_AMT                0
SPECIAL_CONSIDERATIONS    0
ASK_AMT                   0
IS_SUCCESSFUL             0
dtype: int64


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
clean_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [ ]:
clean_df.shape

(34299, 10)

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
at_vc_s = clean_df['APPLICATION_TYPE'].value_counts()
at_vc_s

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced

APP_TYPE_COUNT_CUTOFF = 100
app_type_below_cutoff_s = at_vc_s < APP_TYPE_COUNT_CUTOFF
#print(app_type_below_cutoff_s)

# use the variable name `application_types_to_replace`
application_types_to_replace = at_vc_s[app_type_below_cutoff_s<100].index

# Replace in dataframe
for app in application_types_to_replace:
    clean_df['APPLICATION_TYPE'] = clean_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
clean_df['APPLICATION_TYPE'].value_counts()

Other    34299
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
class_binning_s = clean_df['CLASSIFICATION'].value_counts()
class_binning_s

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >10
counts_class = class_binning_s[class_binning_s>10]
counts_class


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

CLASS_COUNT_CUTOFF = 100
class_below_cutoff_s = class_binning_s < CLASS_COUNT_CUTOFF

classifications_to_replace = list (class_binning_s[class_below_cutoff_s<100].index)
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    clean_df['CLASSIFICATION'] = clean_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
clean_df['CLASSIFICATION'].value_counts()

Other    34299
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
#pd.get_dummies(clean_df)
clean_df = pd.get_dummies(clean_df)
clean_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [ ]:
# Split our preprocessed data into our features and target arrays
y = clean_df['IS_SUCCESSFUL'].values

X = clean_df.drop('IS_SUCCESSFUL', axis=1).values


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3) 

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Compile, train and evaluate

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len( X_train_scaled[0])
hidden_node_1 = 3
hidden_node_2 = 4
hidden_node_3 = 5
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node_1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node_2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 3)                 93        
                                                                 
 dense_10 (Dense)            (None, 4)                 16        
                                                                 
 dense_11 (Dense)            (None, 1)                 5         
                                                                 
Total params: 114
Trainable params: 114
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.15, epochs=100)

Epoch 1/100
684/684 [==============================] - 2s 2ms/step - loss: 0.7188 - accuracy: 0.6350 - val_loss: 0.6430 - val_accuracy: 0.6857
Epoch 2/100
684/684 [==============================] - 1s 2ms/step - loss: 0.6320 - accuracy: 0.6829 - val_loss: 0.6169 - val_accuracy: 0.6997
Epoch 3/100
684/684 [==============================] - 1s 2ms/step - loss: 0.6164 - accuracy: 0.6947 - val_loss: 0.6075 - val_accuracy: 0.6997
Epoch 4/100
684/684 [==============================] - 1s 2ms/step - loss: 0.6099 - accuracy: 0.6969 - val_loss: 0.6033 - val_accuracy: 0.6994
Epoch 5/100
684/684 [==============================] - 2s 2ms/step - loss: 0.6069 - accuracy: 0.6976 - val_loss: 0.6017 - val_accuracy: 0.6989
Epoch 6/100
684/684 [==============================] - 2s 2ms/step - loss: 0.6055 - accuracy: 0.6969 - val_loss: 0.6010 - val_accuracy: 0.6999
Epoch 7/100
684/684 [==============================] - 1s 2ms/step - loss: 0.6047 - accuracy: 0.6971 - val_loss: 0.6006 - val_accuracy: 0.7012

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6065 - accuracy: 0.6975 - 291ms/epoch - 1ms/step
Loss: 0.606468915939331, Accuracy: 0.6974927186965942


In [ ]:
# Export our model to HDF5 file
#from google.colab import files

#nn.save('AlphabetSoupCharity.h5')
#files.download('AlphabetSoupCharity.h5')